# Увеличение производительности библиотеки Pandas: ``eval()`` и ``query()``

Обстракции (транслирование, группировки) Numpy и Pandas весьма производительны, но они зачастую требуют создания временных вспомогательных объектов, что приводит к дополнительным затратам процессорного времени и памяти.
По состоянию на 0,13 от 2014г. библиотека включает эксперементальные инструменты, позволяющие ускорить работу.

## Основания для использования ф-ий ``eval()`` и ``query()``: составные выражения

In [1]:
import numpy as np
import pandas as pd

In [3]:
# векторизованные операции
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
x

array([0.37454012, 0.95071431, 0.73199394, ..., 0.41807198, 0.42867126,
       0.92944855])

In [4]:
len(x)

1000000

In [5]:
%%timeit
x + y

2.51 ms ± 53 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit x + y

2.55 ms ± 55.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit
np.fromiter((xi + yi for xi, yi in zip(x,y)),
            dtype=x.dtype, count=len(x))
# вектроизованные операции быстрее чем списковые включения или списки в ~100раз

212 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
mask = (x > 0.5) & (y < 0.5)
# однако данная обстракция менее эффективна при вычислении составных выражений (так в книге)
# но все течет, все меняется 2.07 ms
# актуальна ли эта тема теперь?

2.07 ms ± 52.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


###### ОКАЗЫВАЕТСЯ ПЕРЕМЕННЫЙ ПОД ОПЕРАТОРОМ ``%%timeit`` ОКАЗЫВАЮТСЯ В ``LOCAL SCOPE`` !!! НЕОЧЕВИДНЫЙ ФАКТ!!!

In [16]:
mask = (x > 0.5) & (y < 0.5)
mask

array([False,  True,  True, ..., False, False, False])

In [17]:
%load_ext memory_profiler
# гразунули memory_profiler получили доступ к %%memit

In [18]:
%%memit
x + y

peak memory: 147.13 MiB, increment: 0.08 MiB


In [19]:
%%memit
mask = (x > 0.5) & (y < 0.5)

peak memory: 149.04 MiB, increment: 1.91 MiB


In [ ]:
# такс...

In [24]:
%%file mprun_x_y.py
import numpy as np
import pandas as pd
def sum_x_y():
    rng = np.random.RandomState(42)
    x = rng.rand(1000000)
    y = rng.rand(1000000)
    x + y

Overwriting mprun_x_y.py


In [40]:
%%file mprun_x_and_y.py
import numpy as np
import pandas as pd
def mask_x_y():
    rng = np.random.RandomState(42)
    x = rng.rand(1000000)
    y = rng.rand(1000000)
    mask = (x > 0.5) & (y < 0.5)

Overwriting mprun_x_and_y.py


In [27]:
from mprun_x_y import sum_x_y

In [31]:
%%mprun -f sum_x_y 
sum_x_y()
# Filename: D:\Python_projects\Jake_VanderPlas\mprun_x_y.py

# Line #    Mem usage    Increment  Occurrences   Line Contents
# =============================================================
#      3    149.4 MiB    149.4 MiB           1   def sum_x_y():
#      4    149.4 MiB      0.0 MiB           1       rng = np.random.RandomState(42)
#      5    157.0 MiB      7.6 MiB           1       x = rng.rand(1000000)
#      6    164.7 MiB      7.6 MiB           1       y = rng.rand(1000000)
#      7    164.7 MiB      0.0 MiB           1       x + y

In [32]:
from mprun_x_and_y import mask_x_y

In [33]:
%%mprun -f mask_x_y
mask_x_y()
# Filename: D:\Python_projects\Jake_VanderPlas\mprun_x_and_y.py

# Line #    Mem usage    Increment  Occurrences   Line Contents
# =============================================================
#      3    149.4 MiB    149.4 MiB           1   def mask_x_y():
#      4    149.4 MiB      0.0 MiB           1       rng = np.random.RandomState(42)
#      5    157.0 MiB      7.6 MiB           1       x = rng.rand(1000000)
#      6    164.7 MiB      7.6 MiB           1       y = rng.rand(1000000)
#      7    163.8 MiB     -0.9 MiB           1       mask = (x > 0.5) & (y < 0.5)

In [34]:
pd.__version__

'1.3.4'

# ВЫВОД: ``pd.__version__ == '1.3.4'`` не имеет указанных недостатков.

In [35]:
import numexpr

In [36]:
%%timeit
mask_numexp = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
# против родных 2.07 ms ± 52.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

2.41 ms ± 31.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [45]:
%%file mprun_x_and_y_msk.py
import numpy as np
import pandas as pd
import numexpr
def mask_numexp_f():
    rng = np.random.RandomState(42)
    x = rng.rand(1000000)
    y = rng.rand(1000000)
    mask_numexp = numexpr.evaluate('(x > 0.5) & (y < 0.5)')

Overwriting mprun_x_and_y_msk.py


In [46]:
from mprun_x_and_y_msk import mask_numexp_f

In [47]:
%%mprun -f mask_numexp_f
mask_numexp_f()
# Filename: D:\Python_projects\Jake_VanderPlas\mprun_x_and_y_msk.py

# Line #    Mem usage    Increment  Occurrences   Line Contents
# =============================================================
#      4    147.8 MiB    147.8 MiB           1   def mask_numexp_f():
#      5    147.8 MiB      0.0 MiB           1       rng = np.random.RandomState(42)
#      6    155.4 MiB      7.6 MiB           1       x = rng.rand(1000000)
#      7    163.0 MiB      7.6 MiB           1       y = rng.rand(1000000)
#      8    164.0 MiB      1.0 MiB           1       mask_numexp = numexpr.evaluate('(x > 0.5) & (y < 0.5)')

In [44]:
%%memit
mask_numexp = numexpr.evaluate('(x > 0.5) & (y < 0.5)')

peak memory: 147.76 MiB, increment: 0.00 MiB


In [ ]:
# короче не пухнет нихрена. вот.

### Использование ф-ии .eval() для эффективных операций

In [48]:
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

In [49]:
%timeit df1 + df2 + df3 + df4

54.8 ms ± 889 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
%timeit pd.eval('df1 + df2 + df3 + df4')
# а тут дает ускорение в 2 раза

26.1 ms ± 895 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [51]:
%memit df1 + df2 + df3 + df4

peak memory: 435.32 MiB, increment: 0.28 MiB


In [52]:
%memit pd.eval('df1 + df2 + df3 + df4')

peak memory: 435.33 MiB, increment: 0.00 MiB


#### Поддерживаемые ф-ией ``pd.eval()`` операции:

In [53]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))

###### Арифметические операторы.

In [54]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

###### Операторы сравнения.

In [55]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

###### Побитовые операторы ``&`` и ``|`` , ``and`` , ``or``

In [56]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In [57]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

######  Атрибуты объектов и индексы.

In [59]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

###### и другие...

### Использование метода ``DataFrame.eval()`` для выполнения операций по столбцам
Возможность ссылаться на столбцы ``по имени``.

In [60]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [61]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

In [62]:
# более локаничная запись
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

###### Мы ``обращались с названиями столбцов`` в вычисляемом выражении как с ``переменными``

### Присваивание в методе ``df.eval()``
``df.eval()`` позволяет выполнять присваивание любому из столбцов.

In [63]:
df

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789
...,...,...,...
995,0.082646,0.036840,0.439733
996,0.008826,0.896578,0.723374
997,0.907270,0.916424,0.978655
998,0.758995,0.535431,0.347766


In [64]:
# создадим вычисляемый столбец
df.eval('D = (A + B) / C', inplace=True)
df

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776
...,...,...,...,...
995,0.082646,0.036840,0.439733,0.271723
996,0.008826,0.896578,0.723374,1.251641
997,0.907270,0.916424,0.978655,1.863469
998,0.758995,0.535431,0.347766,3.722122


In [65]:
# модификация столбца
df.eval('D = (A - B) / D', inplace=True)
df

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.002810
1,0.069087,0.235615,0.154374,-0.084369
2,0.677945,0.433839,0.652324,0.143226
3,0.264038,0.808055,0.347197,-0.176180
4,0.589161,0.252418,0.557789,0.223189
...,...,...,...,...
995,0.082646,0.036840,0.439733,0.168577
996,0.008826,0.896578,0.723374,-0.709271
997,0.907270,0.916424,0.978655,-0.004912
998,0.758995,0.535431,0.347766,0.060064


### Локальные переменные в методе ``df.eval()`` (только в методе, ``не`` в ф-ии ``pd.eval()``)
Символ ``@``отмечает *имя переменной*, а не *имя столбца*, позволяя тем самым вычислять выражения с использованием 2х пространств имен:
- Пространств имен столбцов
- Пространств имен объектов Python

In [66]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

### Метод DataFrame.query()

In [67]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

In [68]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

In [69]:
# позволяет использовать 2 пространства имен
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

# Методы ``.eval`` и ``.query`` следует использовать только для очень больших массивов, инече выигрыш не так очевиден. 